<h1 align=center><font size = 4>Kannada MNIST Identification</font></h1>
<h1 align=center><font size = 5>Image Recognition</font></h1>

# Table of Contents
* [Setup](#setup)
* [Get the Data](#get_data)
* [Explore Kannada MNIST](#explore)
* [Convolutional Neural Networks](#cnn)
* [Make Predictions](#predictions)

<a id="setup"></a>
# Setup

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns
sns.set(style="darkgrid")

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import warnings
warnings.filterwarnings(action="ignore")

<a id="get_data"></a>
# Get the Data

In [ ]:
train_data = pd.read_csv("/kaggle/input/Kannada-MNIST/train.csv")
X_train = train_data.drop(['label'], axis=1).values
y_train = train_data['label'].values

In [ ]:
print(X_train.shape, y_train.shape)

<a id="explore"></a>
# Explore Kannada MNIST

In [ ]:
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

print(np.unique(y_train))
print(np.unique(y_valid))

In [ ]:
def plot_digit(pixels, digit):
    image = pixels.reshape(28, 28)
    plt.imshow(image, cmap = matplotlib.cm.binary, interpolation="nearest")
    plt.title('Digit {}'.format(digit))
    plt.axis("off")
    
plot_digit(X_train[6], y_train[6])

In [ ]:
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = matplotlib.cm.binary, **options)
    plt.axis("off")

In [ ]:
rnd_idx = np.random.permutation(len(X_train))

plt.figure(figsize=(8, 8))
example_images = X_train[rnd_idx[:100]]
plot_digits(example_images, images_per_row=10)
plt.show()

In [ ]:
# reshape and scale data
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_train = X_train / 255

X_valid = X_valid.reshape(X_valid.shape[0], 28, 28, 1)
X_valid = X_valid / 255

<a id="cnn"></a>
# Convolutional Neural Networks

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("cnn.h5", save_best_only=True)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[early_stopping_cb, checkpoint_cb])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.show()

<a id="predictions"></a>
# Make Predictions

In [ ]:
test_data = pd.read_csv("/kaggle/input/Kannada-MNIST/test.csv")
test_data.head()

In [ ]:
model = tf.keras.models.load_model("cnn.h5") # rollback to best model
model.evaluate(X_valid, y_valid)

In [ ]:
X_test = test_data.drop(['id'], axis=1).values
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_test = X_test / 255

y_pred = model.predict(X_test, batch_size=32)

In [ ]:
print(y_pred.shape)
y_pred[:2]

In [ ]:
print(y_pred.shape)
y_pred

In [ ]:
test_data['label'] = np.argmax(y_pred, axis=1)
test_data[['id','label']].head(10)

In [ ]:
idx = 4
label = test_data['label'].values[idx]
some_digit = test_data.drop(['id','label'], axis=1).values[idx].reshape(28, 28)
plt.imshow(some_digit, cmap = matplotlib.cm.binary, interpolation="nearest")
plt.title('Digit {}'.format(label))
plt.axis("off")
plt.show()

In [ ]:
test_data[['id','label']].to_csv('submission.csv', index=False)